In [111]:
from torch import FloatTensor
from torch.autograd import Variable
import torch
import numpy as np

In [86]:
a = Variable(FloatTensor([4,4]))

In [87]:
weights = [Variable(FloatTensor([i,i]), requires_grad=True) for i in (2, 5, 9, 7)]


In [88]:
print(weights)

[tensor([2., 2.], requires_grad=True), tensor([5., 5.], requires_grad=True), tensor([9., 9.], requires_grad=True), tensor([7., 7.], requires_grad=True)]


In [89]:
w1, w2, w3, w4 = weights

In [95]:
b = w1 * a
c = w2 * a
d = w3 * b + w4 * c
L = (10 - d).sum()

In [105]:
x = Variable(torch.randn(3, 3).type(torch.FloatTensor), requires_grad=True)
mat1 = torch.randn(3, 3)
b = x.mm(mat1)
b = torch.inverse(b)
L = b.sum()
L.backward()

In [96]:
L.backward()

In [102]:
for index, weight in enumerate(weights, start=1):
    gradient, *_ = weight.grad.data
    print(f"Gradient of w{index} w.r.t to L: {gradient}")

Gradient of w1 w.r.t to L: -184.0
Gradient of w2 w.r.t to L: -28.0
Gradient of w3 w.r.t to L: -16.0
Gradient of w4 w.r.t to L: -52.0


In [99]:
w1.data[0]

tensor(2.)

In [106]:
x.grad.data

tensor([[ 27.3312,  30.1537,   5.3806],
        [-42.9783, -47.4166,  -8.4610],
        [ 57.8879,  63.8659,  11.3963]])

In [139]:
def kl_divergence(mu1, mu2, sigma_1, sigma_2):
	sigma_diag_1 = np.eye(sigma_1.shape[0]) * sigma_1
	sigma_diag_2 = np.eye(sigma_2.shape[0]) * sigma_2

	sigma_diag_2_inv = np.linalg.inv(sigma_diag_2)

	kl = 0.5 * (np.log(np.linalg.det(sigma_diag_2) / np.linalg.det(sigma_diag_2))
				- mu1.shape[0] + np.trace(np.matmul(sigma_diag_2_inv, sigma_diag_1))
				+ np.matmul(np.matmul(np.transpose(mu2 - mu1), sigma_diag_2_inv), (mu2 - mu1))
				)

	return kl

In [161]:
mu1 = [1,2,3, 4, 5]
var_1 = np.random.randn(5,5)**2
var_1 = (var_1 + var_1.T)/2
res = np.linalg.inv(var_1)
var_1 = np.eye(5)
print(var_1)
var_1 = torch.from_numpy(var_1)
torch.cholesky(var_1)
mu1   = torch.from_numpy(np.array(mu1))

d1 = torch.distributions.MultivariateNormal(mu1, var_1)
d2 = torch.distributions.MultivariateNormal(mu1, var_1)
loss = torch.distributions.kl_divergence(d1, d2).mean()

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


RuntimeError: "triangular_solve_cpu" not implemented for 'Long'

In [142]:
mu1 = np.random.randn(5)
var_1 = np.random.randn(5,5)**2
var_1 = (var_1 + var_1.T)/2

mu2 = np.random.randn(5)
var_2 = np.random.randn(5,5)**2
var_2 = (var_1 + var_1.T)/2

L = kl_divergence(mu1, mu2, var_1, var_2)
L.backward()

AttributeError: 'numpy.float64' object has no attribute 'backward'

In [160]:
B = 5
D = 1
mu1 = torch.rand((B, D), requires_grad=True)
std1 = torch.rand((B, D), requires_grad=True)
p = torch.distributions.Normal(mu1, std1)
mu2 = torch.rand((B, D))
std2 = torch.rand((B, D))
q = torch.distributions.Normal(mu2, std2)

loss = torch.distributions.kl_divergence(p, q).mean()
loss.backward()
mu1.grad.data

tensor([[-3.1342e-01],
        [ 5.9475e-03],
        [-4.3207e-02],
        [-2.5523e+02],
        [ 7.5119e-01]])